In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

import yaml

import pseudopeople as pp
from pseudopeople.utilities import get_configuration
from vivarium.framework.randomness import RandomnessStream
from vivarium.config_tree import ConfigTree

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import datatypes
from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Wed 05 Apr 2023 12:53:08 PM PDT
ndbs
Linux gen-slurm-sexec-p0001 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/pseudopeople_testing


# Find data

```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.3_compressed_hdf/united_states_of_america/2023_04_04_09_18_48/final_results/2023_04_04_16_31_24
```

In [13]:
project_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/v1.3_compressed_hdf'
    '/united_states_of_america/2023_04_04_09_18_48'
    '/final_results/2023_04_04_16_31_24'
)
output_dir = f'{project_dir}/{output_subdir}'

!ls -halt $output_dir

total 364K
drwxrwsr-x  3 rmudambi IHME-Simulationscience 1.0K Apr  4 17:46 ..
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:46 tax_dependents_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:44 tax_1040_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:37 tax_w2_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 social_security_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 wic_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 household_survey_observer_cps
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 household_survey_observer_acs
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 decennial_census_observer
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Apr  4 17:02 .
drwxrwsr-x  2 rmudambi IHME-Simulationscience 334K Apr  4 16:33 logs


In [14]:
!ls -halt $output_dir/decennial_census_observer

total 27G
-rw-r--r--  1 rmudambi IHME-Simulationscience  83M Apr  4 17:23 decennial_census_observer_1299.hdf
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 .
-rw-r--r--  1 rmudambi IHME-Simulationscience  82M Apr  4 17:18 decennial_census_observer_9577.hdf
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Apr  4 17:02 ..
-rw-r--r--  1 rmudambi IHME-Simulationscience  83M Apr  4 16:50 decennial_census_observer_9066.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience  83M Apr  4 16:49 decennial_census_observer_352.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience  83M Apr  4 16:49 decennial_census_observer_3545.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience  83M Apr  4 16:49 decennial_census_observer_3240.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience  82M Apr  4 16:49 decennial_census_observer_3481.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience  82M Apr  4 16:49 decennial_census_observer_5072.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience  82M Apr  4 16:49 de

In [93]:
!ls -halt $output_dir/household_survey_observer_acs

total 362M
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.1M Apr  4 17:23 household_survey_observer_acs_1299.hdf
drwxrwsr-x  2 rmudambi IHME-Simulationscience 167K Apr  4 17:23 .
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.3M Apr  4 17:18 household_survey_observer_acs_9577.hdf
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Apr  4 17:02 ..
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.1M Apr  4 16:50 household_survey_observer_acs_9066.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.1M Apr  4 16:49 household_survey_observer_acs_352.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.1M Apr  4 16:49 household_survey_observer_acs_3545.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.1M Apr  4 16:49 household_survey_observer_acs_3240.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.4M Apr  4 16:49 household_survey_observer_acs_3481.hdf
-rw-r--r--  1 rmudambi IHME-Simulationscience 1.1M Apr  4 16:49 household_survey_observer_acs_5072.hdf
-rw-r--r--  1 rmudambi IHME-Simul

# Choose a seed and define filepaths

In [94]:
seed = 1299
w2_dir = f'{output_dir}/tax_w2_observer'
census_dir = f'{output_dir}/decennial_census_observer'
acs_dir = f'{output_dir}/household_survey_observer_acs'

w2_path = f'{w2_dir}/tax_w2_observer_{seed}.hdf'
census_path = f'{census_dir}/decennial_census_observer_{seed}.hdf'
acs_path = f'{acs_dir}/household_survey_observer_acs_{seed}.hdf'

# Load unnoised census data

In [36]:
%%time
df_census = pd.read_hdf(census_path)
df_census

CPU times: user 23.4 s, sys: 1.63 s, total: 25.1 s
Wall time: 25.3 s


,race_ethnicity,zipcode,age,simulant_id,middle_initial,city,year,street_number,first_name,guardian_2,sex,relation_to_household_head,last_name,state,date_of_birth,guardian_1,street_name,unit_number,housing_type
0,Black,36695,46,1299_0,K,troy,2020,24,Melissa,1299_-1,Female,Reference person,Henry,AL,1974-03-17,1299_-1,coral reef ct s,,Standard
1,Black,36695,5,1299_1,J,troy,2020,24,Levi,1299_-1,Male,Biological child,Henry,AL,2014-08-07,1299_0,coral reef ct s,,Standard
2,White,98012,58,1299_2,J,spokane,2020,300,Michael,1299_-1,Male,Reference person,Herring,WA,1961-04-22,1299_-1,catalpa dr,,Standard
3,White,98012,56,1299_3,K,spokane,2020,300,Mary,1299_-1,Female,Opp-sex spouse,Herring,WA,1963-11-11,1299_-1,catalpa dr,,Standard
4,White,98012,32,1299_4,J,spokane,2020,300,Marquis,1299_-1,Male,Biological child,Herring,WA,1988-01-10,1299_-1,catalpa dr,,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059317,Latino,10028,7,1299_1177788,M,syracuse,2030,1312,Benjamin,1299_1177787,Male,Biological child,Tomlin,NY,2022-06-06,1299_1177786,s dixie hwy,,Standard
2059318,Latino,10028,5,1299_1177789,I,syracuse,2030,1312,Kadyn,1299_1177787,Female,Biological child,Tomlin,NY,2024-09-11,1299_1177786,s dixie hwy,,Standard
2059319,White,90049,48,1299_1177790,N,apple valley,2030,41562,Oscar,1299_-1,Male,Reference person,Hopkins,CA,1981-11-19,1299_-1,oak street,,Standard
2059320,White,90049,43,1299_1177791,M,apple valley,2030,41562,Latasha,1299_-1,Female,Opp-sex spouse,Hopkins,CA,1986-06-04,1299_-1,oak street,,Standard


In [21]:
df_census.dtypes

race_ethnicity                      category
zipcode                               object
age                                    int64
simulant_id                           object
middle_initial                        object
city                                  object
year                                   int64
street_number                         object
first_name                            object
guardian_2                            object
sex                                 category
relation_to_household_head          category
last_name                             object
state                                 object
date_of_birth                 datetime64[ns]
guardian_1                            object
street_name                           object
unit_number                           object
housing_type                        category
dtype: object

In [37]:
%%time
df_census_noisy = pp.generate_decennial_census(census_path)
df_census_noisy

CPU times: user 1min 3s, sys: 2.8 s, total: 1min 5s
Wall time: 1min 7s


,race_ethnicity,zipcode,age,simulant_id,middle_initial,city,year,street_number,first_name,guardian_2,sex,relation_to_household_head,last_name,state,date_of_birth,guardian_1,street_name,unit_number,housing_type
0,Black,36695,46,1299_0,K,troy,2020,24,Melissa,1299_-1,Female,Reference person,Henry,AL,1974-03-17 00:00:00,1299_-1,coral reef ct s,,Standard
1,Black,36695,5,1299_1,J,troy,2020,24,Levi,1299_-1,Male,Biological child,Henry,AL,2014-08-07 00:00:00,1299_0,coral reef ct s,,Standard
2,White,98012,58,1299_2,J,spokane,2020,300,Michael,1299_-1,Male,Reference person,Herring,WA,1961-04-22 00:00:00,1299_-1,catalpa dr,,Standard
3,White,98012,56,1299_3,K,spokane,2020,300,Mary,1299_-1,Female,Opp-sex spouse,Herring,WA,1963-11-11 00:00:00,1299_-1,catalpa dr,,Standard
4,White,98012,32,1299_4,J,spokane,2020,300,Marquis,1299_-1,Male,Biological child,Herring,WA,1988-01-10 00:00:00,1299_-1,catalpa dr,,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059317,Latino,10028,7,1299_1177788,M,syracuse,2030,1312,Benjamin,1299_1177787,Male,Biological child,Tomlin,NY,2022-06-06 00:00:00,1299_1177786,s dixie hwy,,Standard
2059318,Latino,10028,5,1299_1177789,I,syracuse,2030,1312,Kadyn,1299_1177787,Female,Biological child,Tomlin,NY,2024-09-11 00:00:00,1299_1177786,s dixie hwy,,Standard
2059319,White,90049,48,1299_1177790,N,apple valley,2030,41562,Oscar,1299_-1,Male,Reference person,Hopkins,<NA>,1981-11-19 00:00:00,1299_-1,oak street,,Standard
2059320,White,90049,43,1299_1177791,M,apple valley,2030,41562,Latasha,1299_-1,Female,Opp-sex spouse,Hopkins,CA,1986-06-04 00:00:00,1299_-1,oak street,,Standard


# See how many values are missing in the noised and unnoised data

In [38]:
%%time
census_missing = (df_census == '').sum()
census_missing

CPU times: user 2.88 s, sys: 209 ms, total: 3.08 s
Wall time: 3.07 s


race_ethnicity                      0
zipcode                             0
age                                 0
simulant_id                         0
middle_initial                      0
city                                0
year                                0
street_number                   86957
first_name                          0
guardian_2                          0
sex                                 0
relation_to_household_head          0
last_name                           0
state                               0
date_of_birth                       0
guardian_1                          0
street_name                         0
unit_number                   1949936
housing_type                        0
dtype: int64

In [47]:
nc = len(df_census)
nc

2059322

In [39]:
%%time
census_noised = (df_census_noisy != df_census).sum()
census_noised

CPU times: user 8.72 s, sys: 268 ms, total: 8.99 s
Wall time: 8.98 s


race_ethnicity                37866
zipcode                       29028
age                           60849
simulant_id                       0
middle_initial                22660
city                          32754
year                              0
street_number                 45152
first_name                    30257
guardian_2                        0
sex                           30604
relation_to_household_head    39513
last_name                     30968
state                         40581
date_of_birth                 61156
guardian_1                        0
street_name                   33962
unit_number                    2756
housing_type                      0
dtype: int64

In [40]:
census_pct_nonmissing_noised = 100 * census_noised / (df_census != '').sum()
census_pct_nonmissing_noised

race_ethnicity                1.838761
zipcode                       1.409590
age                           2.954807
simulant_id                   0.000000
middle_initial                1.100362
city                          1.590523
year                          0.000000
street_number                 2.289231
first_name                    1.469270
guardian_2                    0.000000
sex                           1.486120
relation_to_household_head    1.918738
last_name                     1.503796
state                         1.970600
date_of_birth                 2.969715
guardian_1                    0.000000
street_name                   1.649184
unit_number                   2.519518
housing_type                  0.000000
dtype: float64

In [29]:
config = get_configuration()
config

decennial_census:
    omission:
        base: 0.0145
            source: initial data
    duplication:
        base: 0.05
            source: initial data
    age:
        missing_data:
            row_noise_level:
                base: 0.01
                    source: initial data
        typographic:
            row_noise_level:
                base: 0.01
                    source: initial data
            token_noise_level:
                base: 0.1
                    source: initial data
            include_original_token_level:
                base: 0.1
                    source: initial data
        age_miswriting:
            row_noise_level:
                base: 0.01
                    source: initial data
            token_noise_level:
                base: 0.1
                    source: initial data
            possible_perturbations:
                1:
                    base: 0.5
                        source: initial data
                -1:
                    bas

In [45]:
age = df_census.age
age_noisy = df_census_noisy.age
age.compare(age_noisy)

,self,other
0,46,46
1,5,5
2,58,58
3,56,56
4,32,32
...,...,...
2059317,7,7
2059318,5,5
2059319,48,48
2059320,43,43


In [42]:
age

0          46
1           5
2          58
3          56
4          32
           ..
2059317     7
2059318     5
2059319    48
2059320    43
2059321    26
Name: age, Length: 2059322, dtype: int64

In [43]:
age_noisy

0          46
1           5
2          58
3          56
4          32
           ..
2059317     7
2059318     5
2059319    48
2059320    43
2059321    26
Name: age, Length: 2059322, dtype: object

In [61]:
age.loc[0] == age_noisy.loc[0]

True

In [60]:
age_noisy.loc[0]

46

In [62]:
age.loc[age != age_noisy]

0          46
1           5
2          58
3          56
4          32
           ..
2059317     7
2059318     5
2059319    48
2059320    43
2059321    26
Name: age, Length: 2059322, dtype: int64

In [35]:
age_noisy.loc[age != age_noisy]

101        <NA>
152        <NA>
212        <NA>
219        <NA>
497        <NA>
           ... 
2059014    <NA>
2059174    <NA>
2059200    <NA>
2059256    <NA>
2059296    <NA>
Name: age, Length: 20717, dtype: object

In [55]:
age_noisy.loc[age_noisy.isna()].map(type).unique()

array([<class 'pandas._libs.missing.NAType'>], dtype=object)

In [56]:
pd.NA

<NA>

In [64]:
config.decennial_census.age

missing_data:
    row_noise_level:
        base: 0.01
            source: initial data
typographic:
    row_noise_level:
        base: 0.01
            source: initial data
    token_noise_level:
        base: 0.1
            source: initial data
    include_original_token_level:
        base: 0.1
            source: initial data
age_miswriting:
    row_noise_level:
        base: 0.01
            source: initial data
    token_noise_level:
        base: 0.1
            source: initial data
    possible_perturbations:
        1:
            base: 0.5
                source: initial data
        -1:
            base: 0.5
                source: initial data

In [65]:
config.decennial_census.age.to_dict()

{'missing_data': {'row_noise_level': 0.01},
 'typographic': {'row_noise_level': 0.01,
  'token_noise_level': 0.1,
  'include_original_token_level': 0.1},
 'age_miswriting': {'row_noise_level': 0.01,
  'token_noise_level': 0.1,
  'possible_perturbations': {1: 0.5, -1: 0.5}}}

In [66]:
# Copied from https://github.com/ihmeuw/pseudopeople/blob/main/tests/unit/test_column_noise.py
RANDOMNESS0 = RandomnessStream(
    key="test_column_noise", clock=lambda: pd.Timestamp("2020-09-01"), seed=0
)

RANDOMNESS1 = RandomnessStream(
    key="test_column_noise", clock=lambda: pd.Timestamp("2020-09-01"), seed=1
)

In [68]:
age_noise_config = config.decennial_census.age.to_dict()
age_noise_config.update({'possible_perturbations': {1: 0.25, 0:0.5, -1: 0.25}})
age_noise_config

{'missing_data': {'row_noise_level': 0.01},
 'typographic': {'row_noise_level': 0.01,
  'token_noise_level': 0.1,
  'include_original_token_level': 0.1},
 'age_miswriting': {'row_noise_level': 0.01,
  'token_noise_level': 0.1,
  'possible_perturbations': {1: 0.5, -1: 0.5}},
 'possible_perturbations': {1: 0.25, 0: 0.5, -1: 0.25}}

In [72]:
ConfigTree(age_noise_config)

missing_data:
    row_noise_level:
        base: 0.01
            source: initial data
typographic:
    row_noise_level:
        base: 0.01
            source: initial data
    token_noise_level:
        base: 0.1
            source: initial data
    include_original_token_level:
        base: 0.1
            source: initial data
age_miswriting:
    row_noise_level:
        base: 0.01
            source: initial data
    token_noise_level:
        base: 0.1
            source: initial data
    possible_perturbations:
        1:
            base: 0.5
                source: initial data
        -1:
            base: 0.5
                source: initial data
possible_perturbations:
    1:
        base: 0.25
            source: initial data
    0:
        base: 0.5
            source: initial data
    -1:
        base: 0.25
            source: initial data

In [73]:
age_copy = age.copy()
age_copy_noisy = pp.noise_functions.miswrite_ages(
    age_copy, ConfigTree(age_noise_config), RANDOMNESS0, 'miswrite_age')
age_copy_noisy

0          45
1           4
2          57
3          57
4          33
           ..
2059317     6
2059318     4
2059319    47
2059320    44
2059321    25
Name: age, Length: 2059322, dtype: object

In [74]:
age_copy.equals(age)

True

In [75]:
age_copy.equals(age_copy_noisy)

False

In [78]:
age_copy_noisy.str.startswith('-').sum()

11965

In [80]:
age_copy_noisy.loc[age_copy_noisy.str.startswith('-')].unique()

array(['-1'], dtype=object)

In [81]:
(age<0).sum()

0

In [89]:
age.loc[age_copy_noisy.astype(int) < 0]

85         0
173        0
174        0
179        0
383        0
          ..
2058872    0
2058873    0
2058876    0
2058877    0
2058879    0
Name: age, Length: 11965, dtype: int64

In [84]:
def int_or_nan(val):
    try:
        return(int(val))
    except:
        return np.nan
age_copy_noisy_int = age_copy_noisy.map(int_or_nan)
age_copy_noisy_int

0          45
1           4
2          57
3          57
4          33
           ..
2059317     6
2059318     4
2059319    47
2059320    44
2059321    25
Name: age, Length: 2059322, dtype: int64

In [85]:
age_copy_noisy_int.isna().any()

False

In [87]:
age_copy_noisy.astype(int)

0          45
1           4
2          57
3          57
4          33
           ..
2059317     6
2059318     4
2059319    47
2059320    44
2059321    25
Name: age, Length: 2059322, dtype: int64

<NA>

In [90]:
!conda list

# packages in environment at /ihme/homes/ndbs/miniconda3/envs/prlpy38:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
altair                    4.2.0                    pypi_0    pypi
anyio                     3.6.2                    pypi_0    pypi
argon2-cffi               21.3.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
asttokens                 2.0.8                    pypi_0    pypi
attrs                     22.1.0                   pypi_0    pypi
babel                     2.10.3                   pypi_0    pypi
backcall                  0.2.0                    pypi_0    pypi
beautifulsoup4            4.11.1                   pypi_0    pypi
bleach                    5.0.1                    pypi_0    pypi
blosc                     1.21.1               h83bc5f

In [95]:
%%time
acs_noisy = pp.generate_american_communities_survey(acs_path)
acs_noisy

CPU times: user 960 ms, sys: 27.7 ms, total: 988 ms
Wall time: 1.1 s


,survey_date,household_id,age,zipcode,simulant_id,middle_initial,city,street_number,first_name,guardian_2,sex,last_name,state,guardian_1,street_name,date_of_birth,unit_number,housing_type
0,2020-04-29,1299_755,92,40222,1299_1813,N,louisville,<NA>,Rose,1299_-1,Female,Hager,KY,1299_-1,amaryllis ct,1927-05-29 00:00:00,,Standard
1,2020-04-29,1299_5030,64,40206,1299_12349,D,louisville,721,Lorie,1299_-1,Female,Maus,KY,1299_-1,clewiston st,1956-02-16 00:00:00,,Standard
2,2020-04-29,1299_5030,68,40206,1299_12350,T,louisville,721,Gregory,1299_-1,Male,Maus,KY,1299_-1,clewiston st,1951-08-31 00:00:00,,Standard
3,2020-04-29,1299_6581,39,24592,1299_16239,C,galax,1214,Marisol,1299_-1,Female,Lange,VA,1299_-1,ardmore avenue,1981-03-31 00:00:00,,Standard
4,2020-04-29,1299_11452,57,93306,1299_28439,L,ramona,564,Angela,1299_-1,Female,Rengifo-Salas,CA,1299_-1,bell st,1962-06-12 00:00:00,,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,2030-05-15,1299_56736,1,02061,1299_1152656,S,medford,2608,Zoe,1299_-1,Female,Estrada,MA,1299_864751,n charleston ave,2028-11-02 00:00:00,,Standard
19357,2030-05-15,1299_557927,0,85338,1299_1170442,M,mesa,328,Daleyza,1299_-1,Female,Reyes,AZ,1299_1138043,washington street,2029-11-01 00:00:00,,Standard
19358,2030-05-15,1299_91102,0,44124,1299_1172772,A,grandview heights,n31w 7151,Madeline,1299_-1,Female,Kroll,LA,1299_592793,w patrick cir,2030-01-07 00:00:00,,Standard
19359,2030-05-15,1299_448701,0,85050,1299_1178514,J,sahuarita,3244,Rosalie,1299_-1,Female,Figueroa,AZ,1299_889928,trinity avenue,2030-04-17 00:00:00,,Standard


In [96]:
pp.generate_american_communities_survey(acs_path, configuration=age_noise_config)

ValueError: Cannot include 0 in age_miswriting.possible_perturbations

In [99]:
pp.noise_entities.NOISE_TYPES.AGE_MISWRITING(age, ConfigTree(age_noise_config), RANDOMNESS0, 'YAK')

ConfigurationKeyError: 'No value at name row_noise_level.'

In [100]:
census_path

'/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.3_compressed_hdf/united_states_of_america/2023_04_04_09_18_48/final_results/2023_04_04_16_31_24/decennial_census_observer/decennial_census_observer_1299.hdf'

In [101]:
acs_path

'/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.3_compressed_hdf/united_states_of_america/2023_04_04_09_18_48/final_results/2023_04_04_16_31_24/household_survey_observer_acs/household_survey_observer_acs_1299.hdf'

In [118]:
age_noise_config = config.decennial_census.age.to_dict()
age_noise_config['age_miswriting'].update({'possible_perturbations': {1: 0.05, 'a': 0.9, -1: 0.05}})
age_noise_config

{'missing_data': {'row_noise_level': 0.01},
 'typographic': {'row_noise_level': 0.01,
  'token_noise_level': 0.1,
  'include_original_token_level': 0.1},
 'age_miswriting': {'row_noise_level': 0.01,
  'token_noise_level': 0.1,
  'possible_perturbations': {1: 0.05, 'a': 0.9, -1: 0.05}}}

In [119]:
df_acs_a = pp.generate_american_communities_survey(acs_path, configuration=age_noise_config)
df_acs_a

,survey_date,household_id,age,zipcode,simulant_id,middle_initial,city,street_number,first_name,guardian_2,sex,last_name,state,guardian_1,street_name,date_of_birth,unit_number,housing_type
0,2020-04-29,1299_755,92,40222,1299_1813,N,louisville,<NA>,Rose,1299_-1,Female,Hager,KY,1299_-1,amaryllis ct,1927-05-29 00:00:00,,Standard
1,2020-04-29,1299_5030,64,40206,1299_12349,D,louisville,721,Lorie,1299_-1,Female,Maus,KY,1299_-1,clewiston st,1956-02-16 00:00:00,,Standard
2,2020-04-29,1299_5030,68,40206,1299_12350,T,louisville,721,Gregory,1299_-1,Male,Maus,KY,1299_-1,clewiston st,1951-08-31 00:00:00,,Standard
3,2020-04-29,1299_6581,39,24592,1299_16239,C,galax,1214,Marisol,1299_-1,Female,Lange,VA,1299_-1,ardmore avenue,1981-03-31 00:00:00,,Standard
4,2020-04-29,1299_11452,57,93306,1299_28439,L,ramona,564,Angela,1299_-1,Female,Rengifo-Salas,CA,1299_-1,bell st,1962-06-12 00:00:00,,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,2030-05-15,1299_56736,1,02061,1299_1152656,S,medford,2608,Zoe,1299_-1,Female,Estrada,MA,1299_864751,n charleston ave,2028-11-02 00:00:00,,Standard
19357,2030-05-15,1299_557927,0,85338,1299_1170442,M,mesa,328,Daleyza,1299_-1,Female,Reyes,AZ,1299_1138043,washington street,2029-11-01 00:00:00,,Standard
19358,2030-05-15,1299_91102,0,44124,1299_1172772,A,grandview heights,n31w 7151,Madeline,1299_-1,Female,Kroll,LA,1299_592793,w patrick cir,2030-01-07 00:00:00,,Standard
19359,2030-05-15,1299_448701,0,85050,1299_1178514,J,sahuarita,3244,Rosalie,1299_-1,Female,Figueroa,AZ,1299_889928,trinity avenue,2030-04-17 00:00:00,,Standard


In [120]:
df_acs_a.age.unique()

array([92, 64, 68, 39, 57, 54, 84, 55, 61, 40, 38, 9, 66, 60, 26, '25',
       70, 78, 73, 63, 42, 15, 24, 22, 80, 71, 48, 18, 52, 28, 67, 31, 11,
       10, 7, '74', 77, 53, 17, 50, 91, '33', 20, 51, 75, 21, 49, 16, 34,
       35, 5, 3, 59, 45, 19, 81, 83, <NA>, 56, 36, 37, 6, 58, 69, 43, 14,
       76, 93, '29', 0, 47, 33, 29, 4, 1, 12, 62, 88, 46, 23, 25, 8, 44,
       65, 13, 32, 27, '19', 82, '16', 2, 87, 41, 86, 72, 30, '58', 74,
       '48', '37', 89, 79, '59', '15', '43', '45', '2', 85, '60', '31',
       95, '75', '70', '6', '81', '14', '3', '12', 90, '72', '26', 97,
       '63', '62', '20', '41', '56', '21', '49', 94, '10', '44', 96, '1',
       '27', '38', '18', '28', '67', '4', '77', '84', '82', '68', '88',
       '65', '55', '13', '5', '54', '39', '52', '47', '35', '42', '85',
       98, '7', '50', '9', '22', '320', '79', '57', '24', '61', '8', '34',
       '66', '17', '30', '743', '83', '90', '23', '11', 99, '40', '80',
       '69', '32', 100, '96', '635', '89', '51', '71

In [115]:
df_acs_a.age.value_counts()

11     296
41     266
59     265
40     263
15     263
      ... 
96       1
100      1
320      1
84       1
104      1
Name: age, Length: 196, dtype: int64

In [121]:
df_acs_a2 = pp.generate_american_communities_survey(acs_path, configuration={'age': age_noise_config})
df_acs_a2

,survey_date,household_id,age,zipcode,simulant_id,middle_initial,city,street_number,first_name,guardian_2,sex,last_name,state,guardian_1,street_name,date_of_birth,unit_number,housing_type
0,2020-04-29,1299_755,92,40222,1299_1813,N,louisville,<NA>,Rose,1299_-1,Female,Hager,KY,1299_-1,amaryllis ct,1927-05-29 00:00:00,,Standard
1,2020-04-29,1299_5030,64,40206,1299_12349,D,louisville,721,Lorie,1299_-1,Female,Maus,KY,1299_-1,clewiston st,1956-02-16 00:00:00,,Standard
2,2020-04-29,1299_5030,68,40206,1299_12350,T,louisville,721,Gregory,1299_-1,Male,Maus,KY,1299_-1,clewiston st,1951-08-31 00:00:00,,Standard
3,2020-04-29,1299_6581,39,24592,1299_16239,C,galax,1214,Marisol,1299_-1,Female,Lange,VA,1299_-1,ardmore avenue,1981-03-31 00:00:00,,Standard
4,2020-04-29,1299_11452,57,93306,1299_28439,L,ramona,564,Angela,1299_-1,Female,Rengifo-Salas,CA,1299_-1,bell st,1962-06-12 00:00:00,,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,2030-05-15,1299_56736,1,02061,1299_1152656,S,medford,2608,Zoe,1299_-1,Female,Estrada,MA,1299_864751,n charleston ave,2028-11-02 00:00:00,,Standard
19357,2030-05-15,1299_557927,0,85338,1299_1170442,M,mesa,328,Daleyza,1299_-1,Female,Reyes,AZ,1299_1138043,washington street,2029-11-01 00:00:00,,Standard
19358,2030-05-15,1299_91102,0,44124,1299_1172772,A,grandview heights,n31w 7151,Madeline,1299_-1,Female,Kroll,LA,1299_592793,w patrick cir,2030-01-07 00:00:00,,Standard
19359,2030-05-15,1299_448701,0,85050,1299_1178514,J,sahuarita,3244,Rosalie,1299_-1,Female,Figueroa,AZ,1299_889928,trinity avenue,2030-04-17 00:00:00,,Standard


In [122]:
df_acs_a3 = pp.generate_american_communities_survey(
    acs_path,
    configuration={'american_communities_survey':{'age': age_noise_config}})
df_acs_a3

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U21')) -> None